<a href="https://colab.research.google.com/github/GeorgeShishkanov/university_project/blob/main/Text_classification_with_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing libraries

In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

In [35]:
import pandas as pd
import numpy as np
import re
import datasets
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
from sklearn.model_selection import train_test_split
import transformers
from transformers import Trainer
from transformers import TrainingArguments
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    PretrainedConfig,
    default_data_collator   
)

# Data preprocessing

In [4]:
!unzip /content/train.csv.zip

Archive:  /content/train.csv.zip
  inflating: train.csv               


In [5]:
test_data = pd.read_csv("/content/test.csv")
train_data = pd.read_csv("/content/train.csv")
sub_data = pd.read_csv("/content/sample_submission.csv")

In [6]:
def text_cleaner(text, remove_digits=True):
    text = re.sub('http://\S+|https://\S+|www.\S+', '', text)
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text.strip()

In [7]:
train_data = train_data.drop(train_data.columns[0],axis=1)
train_data = train_data.dropna()

In [8]:
train_data['Text'] = train_data['Text'].apply(text_cleaner)

In [9]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()

train_data['Sentiment'] = label_encoder.fit_transform(train_data['Sentiment'])

train_data['Sentiment'].unique()
print(label_encoder.classes_)

['Extremely Negative' 'Extremely Positive' 'Negative' 'Neutral' 'Positive']


In [10]:
lst1 = list(label_encoder.classes_)
lst2 = [3, 4, 0, 2, 1]
label_to_id= dict(zip(lst1, lst2))

In [11]:
test_data = test_data.dropna()

In [12]:
train_data = train_data.rename(columns={'Text':'text', 'Sentiment':'label'})

In [13]:
test_data['id1'] = test_data['id']
test_data = test_data.drop('id', axis=1)
test_data = test_data.rename(columns={'Text':'text', 'id1':'id'})

In [14]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)

In [15]:
train_dataset

Dataset({
    features: ['text', 'label', '__index_level_0__'],
    num_rows: 41155
})

#Tokenizer


In [16]:
from transformers import AutoTokenizer


checkpoint = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [17]:
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)

In [18]:
tokenized_datasets = train_dataset.map(tokenize_function)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

tokenized_datasets

  0%|          | 0/41155 [00:00<?, ?ex/s]

Dataset({
    features: ['text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 41155
})

#Model


In [19]:
config = AutoConfig.from_pretrained("bert-base-uncased", num_labels=5)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased",config=config,ignore_mismatched_sizes=True)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

#Func for metrics

In [20]:
!pip install evaluate
import evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 1.7 MB/s 


In [21]:
def metrics(eval_preds):
    metric = evaluate.load("accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

#Train Model

In [22]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [33]:
training_args = TrainingArguments(
    "test-trainer",     
    evaluation_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_steps=1,
    optim='adamw_torch'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=metrics
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [31]:
model.config.label2id = label_to_id
model.config.id2label = {id: label for label, id in config.label2id.items()}

In [25]:
tokenized_datasets

Dataset({
    features: ['text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 41155
})

In [27]:
split_data= train_dataset.train_test_split(test_size=0.2)
split_data

DatasetDict({
    train: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 32924
    })
    test: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 8231
    })
})

In [28]:
tokenized_train = split_data['train'].map(tokenize_function, batched=True)
tokenized_val = split_data['test'].map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/33 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

In [36]:

trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 41155
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3861
  Number of trainable parameters = 109486085


Epoch,Training Loss,Validation Loss


Saving model checkpoint to test-trainer/checkpoint-500
Configuration saved in test-trainer/checkpoint-500/config.json
Model weights saved in test-trainer/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-500/special_tokens_map.json


Epoch,Training Loss,Validation Loss,Accuracy
1,0.043200,0.208023,0.929656
2,0.315400,0.087647,0.976188
3,0.028200,0.087647,0.976188


Saving model checkpoint to test-trainer/checkpoint-1000
Configuration saved in test-trainer/checkpoint-1000/config.json
Model weights saved in test-trainer/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-1000/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 8231
  Batch size = 16


Epoch,Training Loss,Validation Loss


Saving model checkpoint to test-trainer/checkpoint-1500
Configuration saved in test-trainer/checkpoint-1500/config.json
Model weights saved in test-trainer/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-1500/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-1500/special_tokens_map.json
Saving model checkpoint to test-trainer/checkpoint-2000
Configuration saved in test-trainer/checkpoint-2000/config.json
Model weights saved in test-trainer/checkpoint-2000/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-2000/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-2000/special_tokens_map.json
Saving model checkpoint to test-trainer/checkpoint-2500
Configuration saved in test-trainer/checkpoint-2500/config.json
Model weights saved in test-trainer/checkpoint-2500/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-2500/tokenizer_config.json
Special tokens file 

TrainOutput(global_step=3861, training_loss=0.19815529781357283, metrics={'train_runtime': 1505.7008, 'train_samples_per_second': 81.998, 'train_steps_per_second': 2.564, 'total_flos': 3923848482700446.0, 'train_loss': 0.19815529781357283, 'epoch': 3.0})

In [37]:
def get_prediction(text):
    inputs = tokenizer(text, truncation=True,padding=True, return_tensors="pt").to("cuda")
    outputs = model(**inputs)
    probs = outputs[0].softmax(1)
    return model.config.id2label[probs.argmax().item()]

In [38]:
tokenized_test = test_dataset.map(tokenize_function, batched=True)
trainer.evaluate(tokenized_test)

  0%|          | 0/4 [00:00<?, ?ba/s]

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, id, __index_level_0__. If text, id, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3798
  Batch size = 16


{'eval_runtime': 19.4215,
 'eval_samples_per_second': 195.557,
 'eval_steps_per_second': 12.254,
 'epoch': 3.0}

In [43]:
test_data['Sentiment'] = test_data['text'].apply(lambda text: get_prediction(text))
test_data.head()

,text,id,Sentiment
0,TRENDING: New Yorkers encounter empty supermar...,787bc85b-20d4-46d8-84a0-562a2527f684,Neutral
1,When I couldn't find hand sanitizer at Fred Me...,17e934cd-ba94-4d4f-9ac0-ead202abe241,Extremely Positive
2,Find out how you can protect yourself and love...,5914534b-2b0f-4de8-bb8a-e25587697e0d,Positive
3,#Panic buying hits #NewYork City as anxious sh...,cdf06cfe-29ae-48ee-ac6d-be448103ba45,Neutral
4,#toiletpaper #dunnypaper #coronavirus #coronav...,aff63979-0256-4fb9-a2d9-86a3d3ca5470,Extremely Negative


In [44]:
submission = test_data[['id', 'Sentiment']]
submission.to_csv(('submission_kaggle2.csv'), index=False)